In [46]:
import requests # from urllib.request import urlopen
from bs4 import BeautifulSoup # HTTP Response -> HTML 

import pandas as pd

from datetime import datetime

import time 
import re #정규 표현식 활용

In [47]:
#망고플레이트에서 음식을 키워드로 맛집 리스트 가져오는 function
def get_rest_list(keyword):
    
    #페이지, 맛집리스트 초기화
    pagenum = 1
    restaurant_list = []
    
    #page별로 while 루프 돌려서 상호명 뽑아오기
    while pagenum <= 2: 
        try:
            url = 'https://www.mangoplate.com/search/?keyword='+keyword+'&page='+str(pagenum)#keyword를 input으로 망고플레이트 url 생성

            session = requests.Session()
            headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
            }
            html = session.get(url, headers=headers).content 
            source = BeautifulSoup(html, 'html.parser')

            restaurants = source.find_all('h2', {'class' : 'title'}) #h2태그의 class 'title' => 상호명

            for restaurant in restaurants:#텍스트 깔끔하게 처리 및 리스트에 상호명 추가 루프

                n_restaurant = re.sub('\s', '', restaurant.get_text()) #텍스트 중간에 빈칸 없애기
                restaurant_list.append(n_restaurant)#리스트에 상호명 추가
        except:
            print(pagenum,'페이지 까지의 결과입니다.')
        
        pagenum += 1
        
    return restaurant_list #keyword 관련 맛집 리스트 생성

In [48]:
cnoodlelist = get_rest_list('냉면') #냉면 맛집 리스트 생성

In [49]:
cnoodlelist

['허브족발(본점)',
 '윤밀원',
 '김진목삼',
 '진미평양냉면',
 '서관면옥',
 '광릉한옥집',
 '광화문국밥',
 '춘향미엔',
 '우래옥(을지로점)',
 '을지면옥',
 '설눈',
 '봉밀가',
 '봉피양(방이점)',
 '미엔아이',
 '윤선희의평양냉면양각도',
 '청기와타운',
 '서평면옥',
 '몽탄',
 '정인면옥',
 '강서면옥',
 '오장동흥남집(본점)',
 '거대갈비',
 '평안도상원냉면',
 '명지원이동갈비',
 '벽제갈비(신촌점)',
 '공단떡볶이',
 '고명집',
 '우밀면옥',
 '단천식당',
 '명인등심(압구정점)',
 '평남면옥',
 '서정가야밀면',
 '더진미평냉',
 '명인등심(삼성직영점)',
 '영동냉면',
 '을밀대(본점)',
 '능라도(강남점)',
 '능라도(분당점)',
 '외래향(서울대점)',
 '옥돌현옥']

In [50]:
def get_mango_info(restaurant):#상호명을 input으로 관련 정보 받아오는 function

    mango_info = {} #빈 dict 생성

    url = 'https://www.mangoplate.com/search/?keyword='+restaurant #상호명으로 망고플레이트 검색 url

    session = requests.Session()
    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    html = session.get(url, headers=headers).content 
    source = BeautifulSoup(html, 'html.parser')

    rest_code = source.find('a',{'class':'only-desktop_not'}).attrs['href']#검색결과에서 첫번째 결과의 고유 코드 따오기 
    newurl = 'https://www.mangoplate.com'+rest_code #고유 코드 활용한 url 주소 생성
    newhtml = session.get(newurl, headers=headers).content 
    newsource = BeautifulSoup(newhtml, 'html.parser')
    
    rate = float(re.sub('\s','',newsource.find('strong',{'class':'rate-point'}).get_text()))#평점이 속한 태그에서 텍스트 가져와 float로 바꾸기 (나중에 계산 용이)

    rest_table = newsource.find('table',{'class':'info'})#관련 정보가 있는 테이블 경로

    address = rest_table.find_all('td')[0].find_all('span')[1].get_text()#주소
    phone = rest_table.find_all('td')[1].get_text()#전화번호
    kind = re.sub('\s','',rest_table.find_all('td')[2].get_text())#음식 종류(정규표현식 활용하여 깔끔하게 정리)
    price = rest_table.find_all('td')[3].get_text()#가격대

    #각각의 정보를 만든 dict의 key값의 value로 추가
    mango_info['name'] = restaurant
    mango_info['rate'] = rate
    mango_info['address'] = address
    mango_info['phone'] = phone
    mango_info['kind'] = kind
    mango_info['price'] = price
    
    return mango_info #하나의 dict를 최종 값으로 return



In [55]:
print(get_mango_info('윤밀원'))

{'name': '윤밀원', 'rate': 4.6, 'address': '경기도 성남시 분당구 정자동 223-9', 'phone': '031-714-8388', 'kind': '국수/면요리', 'price': '만원 미만'}


In [51]:
def get_naver_info(restaurant): #네이버 평점 및 리뷰 가져오기
    
    naver_info = {}
    regex = "\(.*\)|\s-\s.*"
    query = re.sub(regex, '', restaurant)
    
    url = "https://search.naver.com/search.naver?where=nexearch&sm=tab_jum&query=" + query
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')
    
    try:
        reviewcount = int(source.find_all('span', {'class': 'count'})[1].get_text())
        rate = float(source.find_all('div', {'class': 'star_area'})[0].find('em').get_text())
    except:
        reviewcount = 'N/A'
        rate = 'N/A'
    
    naver_info['reviewcount'] = reviewcount
    naver_info['rate'] = rate
    
    return naver_info


    

In [54]:
print(get_naver_info('윤밀원'))

{'reviewcount': 'N/A', 'rate': 'N/A'}


In [52]:
def get_dining_info(restaurant):
    
    dining_info = {}
    review_list = []
    regex = "\(.*\)|\s-\s.*"
    query = re.sub(regex, '', restaurant)
    
    url = 'https://www.diningcode.com/isearch.php?query=' + query
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')
    try:
        ul_tag = source.find('div', {'id':'div_rn'})
        li_tag = ul_tag.find('li')
        a_tag = li_tag.find('a')
        target_url = 'https://www.diningcode.com' + a_tag.attrs['href']
        target_url

        web_review = requests.get(target_url).content
        source_review = BeautifulSoup(web_review, 'html.parser')

        #평균 평점 꺼내기
        p_tag = source_review.find('p', {'class':'grade'})
        # p_tag
        review_point = int(p_tag.find('strong').get_text().replace('점',''))

        #평균 별점 꺼내기
        star_point = source_review.find('p', {'id':'lbl_star_point'}).get_text()
        star_point = star_point.replace("\n", "")
        star_point = float(star_point.replace('점',''))# 교체하다

        reviews = source_review.find_all('p',{'class':'review_contents btxt'})

        for review in reviews:
            review_list.append(re.sub('\n','',review.get_text()))

        dining_info['review_point'] = review_point
        dining_info['star_point'] = star_point
        dining_info['review'] = review_list
                               
    except:
        dining_info['review_point'] = 'N/A'
        dining_info['star_point'] = 'N/A'
        dining_info['review'] = 'N/A'
        
    return dining_info

In [53]:
print(get_dining_info('윤밀원'))

{'review_point': 67, 'star_point': 4.7, 'review': ['한글날 딸들을 데리고 분당 정자동 윤밀원으로  평양냉면 양곰탕 양지국수를 주문 평양냉면 면발 매밀함량 굿 식감 굿 육수는 동치미맛이 짙음 양지국수 먹어보지 못했던 특색있는 깊고 칼칼한 맛 굿 양 곰탕은 고기냄새는잘 잡았으나 국물은 그다지 깨끗하게 개운하지 못했음 다음엔 족발을 꼭 먹어봐야겠다 주택가에 위치해 가로수들이 단풍이들어 가을 정취를 도심에서 느낄수있어 더더욱 좋았던 가족나들이였습니다', '족발은 요즘 유행하는 스타일이고 제공되는 고수와 소스등으로 깔끔함. 평양냉면 역시 입문자가 먹어도 무난한 맛. 칼국수는 개인적으로 별로였습니다. 대기가 길지만 일단 입장하면 음식도 빨리 나오고 서비스도 좋아서 기분 좋게 식사 가능합니다. 주차는 근처 공영 주차장 추천합니다.', '첫번째 사진 : 양지칼국수두번째 사진 : 막국수세번째 사진 : 냉면칼국수 종류(양지칼국수, 칼국수)는 흰국물과 빨간국물 두가지 종류가 있다고 한다. 양지칼국수 흰국물로 시켰는데 순하고 담백한 맛이었다.막국수는 막국수보단 비빔냉면 같은 느낌이었다. 자극적이지 않으며 얘 또한 순하고 단백한 맛이었다.냉면은 평양냉면 스타일같고, 순한 맛이었다.총평 : 멀리갈게 아니라면 아주 괜찮은 선택지가 될거같다. 재방문의사 있음.양이 상당히 많다. 대식가가 아니라면 인원수보다 적게 시키는데 좋을거같으며, 족발을 시키는 경우에는 더 적게 시켜도 될거 같다. 전체적으로 고급지고 단백하고 순한 맛으로 어른들이 좋아할것 같고, 아이들도 칼국수와 막국수는 좋아할것 같다. 메뉴가 전체적으로 순한맛이라 막국수도 자극적이지 않아서 어린 아이들이 먹기 좋을거같다. 재방문의사 있음.', '근처에 공영 주차장이 있어서 거기에 차 대시면 됩니다. 막국수가 진짜 부드럽게 녹아요. 맛있어요. 족발도 잡내 없고 정말 맛있네요.', '나이와 상관없이 맛난 음식을 즐길수있는 행복한 장소']}


In [56]:
#DataFrame을 위한 빈 리스트 생성

mango_name = []
mango_rate = []
mango_address = []
mango_phone = []
mango_kind = []
mango_price = []
naver_reviewcount = []
naver_rate = []
dining_review_point = []
dining_star_point = []
dining_review = []

#음식정 상호명 리스트를 활용하여 각각의 점포의 정보를 받아오는 for loop
for restaurant in cnoodlelist:
    
    mango_info = get_mango_info(restaurant)
    naver_info = get_naver_info(restaurant)
    dining_info = get_dining_info(restaurant)
    #위에서 정의한 망고플레이트 음식점 정보를 하나의 dict로 가져옴
    
    #각각의 리스트에 고유 정보 추가
    mango_name.append(mango_info['name'])
    mango_rate.append(mango_info['rate'])
    mango_address.append(mango_info['address'])
    mango_phone.append(mango_info['phone'])
    mango_kind.append(mango_info['kind'])
    mango_price.append(mango_info['price'])
    naver_reviewcount.append(naver_info['reviewcount'])
    naver_rate.append(naver_info['rate'])
    dining_review_point.append(dining_info['review_point'])
    dining_star_point.append(dining_info['star_point'])
    dining_review.append(dining_info['review'])

    

In [57]:
#만든 리스트로 DataFrame 생성

restaurant_df = pd.DataFrame({'음식점':mango_name,
                         '망고평점': mango_rate,
                         '네이버평점': naver_rate,
                         '네이버리뷰수': naver_reviewcount,
                         '다이닝코드평점': dining_review_point,
                         '다이닝코드별점': dining_star_point,
                         '다이닝코드리뷰': dining_review,     
                         '주소': mango_address,
                         '전화': mango_phone,
                         '종류': mango_kind,
                         '가격': mango_price})

restaurant_df

,음식점,망고평점,네이버평점,네이버리뷰수,다이닝코드평점,다이닝코드별점,다이닝코드리뷰,주소,전화,종류,가격
0,허브족발(본점),4.6,N/A,N/A,67,4.3,[족발 입문자에게 데려오면 곧바로 족발보쌈중 족발파가 될 수 있는 곳. 오히려 입맛...,서울시 영등포구 당산동6가 237-38,02-2633-4339,고기요리,3만원-4만원
1,윤밀원,4.6,N/A,N/A,67,4.7,[한글날 딸들을 데리고 분당 정자동 윤밀원으로 평양냉면 양곰탕 양지국수를 주문 평...,경기도 성남시 분당구 정자동 223-9,031-714-8388,국수/면요리,만원 미만
2,김진목삼,4.6,4.69,246,55,4.9,[주차할 수 없는 서촌 골목에 위치. 1호점과 2호점이 아주 가깝게 붙어있는데 두 ...,서울시 종로구 필운동 154,02-929-2929,고기요리,만원-2만원
3,진미평양냉면,4.5,N/A,N/A,79,4,[발렛비 2천원은 후불이에요. 평양냉면은 솔직히 기대 이하였어요... 육향이 다른 ...,서울시 강남구 논현동 115-10,02-515-3469,국수/면요리,만원-2만원
4,서관면옥,4.5,N/A,N/A,59,3.9,[아름다운 한 상.맛을 떠나 조선시대 면반상을 재현한 노고에 감사드린다.손이 엄청 ...,서울시 서초구 서초동 1675-5,02-521-9945,국수/면요리,만원-2만원
5,광릉한옥집,4.5,4.43,613,61,4.1,[동네에서 나름 유명한 곳이다 원래는 간판없는 가계로 유명했던장소인데 속사정은 모르...,경기도 남양주시 진접읍 부평리 602-1,031-574-6630,고기요리,만원-2만원
6,광화문국밥,4.5,N/A,N/A,70,4.2,[오랜만에 찾은 맛집. 국물이 깔끔하고 맛있음. 밑반찬과 잘 어울림. 왜 지금에서야...,서울시 중구 정동 1-48,02-738-5688,탕/찌개/전골,만원-2만원
7,춘향미엔,4.5,N/A,N/A,56,4,[맛집이라 해서 찾아갔어요~ 문앞에 차량 세웠는에 유료주차장으로 가라고 하네요. 흠...,서울시 성동구 행당동 299-1,02-2281-9031,딤섬/만두,만원 미만
8,우래옥(을지로점),4.4,N/A,N/A,88,4.1,"[을지로 평냉맛집~맛은 워낙 유명해서 설명이 필요없는 곳 기본웨이팅 있으며, 주차가...",서울시 중구 주교동 118-1,02-2265-0151,국수/면요리,만원-2만원
9,을지면옥,4.4,4.35,944,84,4.1,[역시 물냉면 전문입니다. 너무 슴슴하다 싶으시면 비빔을 드시면 되고냉면이 싫으면 ...,서울시 중구 입정동 161,02-2266-7052,국수/면요리,만원-2만원


In [58]:
restaurant_df.head(3)

,음식점,망고평점,네이버평점,네이버리뷰수,다이닝코드평점,다이닝코드별점,다이닝코드리뷰,주소,전화,종류,가격
0,허브족발(본점),4.6,N/A,N/A,67,4.3,[족발 입문자에게 데려오면 곧바로 족발보쌈중 족발파가 될 수 있는 곳. 오히려 입맛...,서울시 영등포구 당산동6가 237-38,02-2633-4339,고기요리,3만원-4만원
1,윤밀원,4.6,N/A,N/A,67,4.7,[한글날 딸들을 데리고 분당 정자동 윤밀원으로 평양냉면 양곰탕 양지국수를 주문 평...,경기도 성남시 분당구 정자동 223-9,031-714-8388,국수/면요리,만원 미만
2,김진목삼,4.6,4.69,246,55,4.9,[주차할 수 없는 서촌 골목에 위치. 1호점과 2호점이 아주 가깝게 붙어있는데 두 ...,서울시 종로구 필운동 154,02-929-2929,고기요리,만원-2만원


In [59]:
restaurant_df.to_excel('result_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')